In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install -U safetensors
!pip install -U bitsandbytes
!pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# HuggingFaceにログイン
os.environ["HF_TOKEN"] = 

# オフロードフォルダの設定
offload_folder = "/tmp/model_offload"
os.makedirs(offload_folder, exist_ok=True)

# 量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

# トークナイザーのロード - padding_side と pad_token の設定を追加
tokenizer = AutoTokenizer.from_pretrained(
   #"elyza/ELYZA-japanese-Llama-2-7b",
    "meta-llama/Meta-Llama-3-8B",
    token=os.environ["HF_TOKEN"],
    padding_side="right",  # パディングの位置を右側に設定
)

# Llama-3ではデフォルトでpad_tokenが設定されていないので追加
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# モデルのロード
model = AutoModelForCausalLM.from_pretrained(
    #"elyza/ELYZA-japanese-Llama-2-7b",
    "meta-llama/Meta-Llama-3-8B",
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    token=os.environ["HF_TOKEN"],
    offload_folder=offload_folder
)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [26]:
# 40
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
[選択肢]:[ア，イ，ウ]
[答え]:"""

for i in range(3):
  model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
  input_ids = model_input["input_ids"]

  with torch.no_grad():
    result = model.generate(
                input_ids,
                max_new_tokens=300,
                # eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
            )
    result = result[0][input_ids.shape[-1]:]
    output = tokenizer.decode(result, skip_special_tokens=True)
    print("\n-----生成結果-----\n", output.split("[問題]")[0])

    del input_ids
    del model_input
    torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
 [ア，ウ，イ]




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
 [ウ，イ，ア]



-----生成結果-----
 [ウ，イ，ア]


In [28]:
# 41
prompt = """
[問題]に対する[答え]を[選択肢]のア～ウを年代の古い順に正しく並べよ。
[問題]:日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

[選択肢]:[ア，イ，ウ]
[答え]:解答: ウ→イ→ア
[問題]:江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

[選択肢]:[ア，イ，ウ]
[答え]:ウ→ア→イ
[問題]:中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

[選択肢]:[ア，イ，ウ]
[答え]:イ→ア→ウ
[問題]:加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

[選択肢]:[ア，イ，ウ]
[答え]:ウ→ア→イ
[問題]:9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。


[選択肢]:[ア，イ，ウ]
[答え]:"""

for i in range(3):
  model_input = tokenizer(prompt, return_tensors="pt").to(model.device)
  input_ids = model_input["input_ids"]

  with torch.no_grad():
    result = model.generate(
                input_ids,
                max_new_tokens=300,
                # eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
            )
    result = result[0][input_ids.shape[-1]:]
    output = tokenizer.decode(result, skip_special_tokens=True)
    print("\n-----生成結果-----\n", output.split("[問題]")[0])

    del input_ids
    del model_input
    torch.cuda.empty_cache()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
 ウ→イ→ア



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----生成結果-----
 ウ→ア→イ


-----生成結果-----
 ウ→ア→イ



In [29]:
# 42

import pandas as pd
import torch
import re
from tqdm.notebook import tqdm
import os

# JMMLUリポジトリのクローン（まだ存在しない場合）
if not os.path.exists("./JMMLU"):
    print("JMMLUリポジトリをクローン中...")
    !git clone https://github.com/nlp-waseda/JMMLU.git

def generate_prompt(question, choices):
    """プロンプトを生成する関数"""
    prompt = f"""以下の問題に対して、選択肢A、B、C、Dから最も適切な答えを1つ選んでください。
選択肢を選ぶ理由を簡潔に説明し、最後に「答え: 」の後に選んだ選択肢（A、B、C、Dのいずれか）のみを記入してください。

問題: {question}

選択肢:
A. {choices[0]}
B. {choices[1]}
C. {choices[2]}
D. {choices[3]}

"""
    return prompt

def extract_answer(text):
    """モデルの出力から回答を抽出する関数"""
    # パターン1: 「答え: X」という形式
    pattern1 = r"答え\s*[:：]\s*([ABCD])"
    match1 = re.search(pattern1, text)
    if match1:
        return match1.group(1)

    # パターン2: 最後の行に単独で「A」「B」「C」「D」のいずれかがある
    pattern2 = r"([ABCD])\s*$"
    match2 = re.search(pattern2, text)
    if match2:
        return match2.group(1)

    # パターン3: 文中に「選択肢はX」「Xが正解」などの表現がある
    pattern3 = r"選択肢[はがのはABCD]*\s*([ABCD])"
    match3 = re.search(pattern3, text)
    if match3:
        return match3.group(1)

    # テキスト全体からA,B,C,Dのみを探す
    choices = re.findall(r'\b[ABCD]\b', text)
    if choices:
        # 最も頻繁に出現する選択肢を回答とする
        return max(set(choices), key=choices.count)

    return None

def evaluate_jmmlu_computer_security(model, tokenizer):
    """JMMLUのコンピュータセキュリティデータセットを評価する関数"""
    # CSVファイルのパス
    csv_path = "./JMMLU/JMMLU/computer_security.csv"

    # CSVファイル読み込み
    print(f"コンピュータセキュリティCSV読み込み中: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"データセット読み込み完了。問題数: {len(df)}問")

    # カラム名の確認
    print(f"カラム名: {df.columns.tolist()}")

    # データ形式の確認（最初の行）
    first_row = df.iloc[0].tolist()
    print("\n最初の問題のデータ:")
    for i, value in enumerate(first_row):
        print(f"列{i}: {value}")

    # 評価用データの準備
    correct = 0
    total = 0

    # 進捗表示用
    print("\n評価開始 - 全99問のコンピュータセキュリティ問題")

    # 各問題に対して評価
    for i in tqdm(range(len(df)), desc="Evaluating"):
        row = df.iloc[i].tolist()

        # 行の形式に応じて問題と選択肢を抽出
        question = row[0]  # 最初の列を問題文とする
        choices = row[1:5]  # 2-5列目を選択肢とする
        correct_answer = row[5]  # 6列目を正解とする

        # プロンプト生成
        prompt = generate_prompt(question, choices)

        # トークナイズとモデル入力の準備
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # 生成
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=0.1,
                do_sample=True,
                top_p=0.95,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id
            )

        # 出力のデコード
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # プロンプト部分を削除して回答のみを取得
        predicted_text = generated_text[len(prompt):]

        # 回答の抽出
        predicted_answer = extract_answer(predicted_text)

        # 正解判定
        is_correct = (predicted_answer == correct_answer)
        if is_correct:
            correct += 1
        total += 1

        # 25問ごとに進捗状況を表示
        if (i + 1) % 25 == 0:
            print(f"進捗: {i+1}/99問完了、現在の正解率: {correct}/{total} ({correct/total:.4f})")

    # 最終結果の表示
    accuracy = correct / total
    print(f"\n評価完了！")
    print(f"モデル: elyza/ELYZA-japanese-Llama-2-7b")
    print(f"データセット: JMMLU computer_security")
    print(f"正解数: {correct}/{total}問")
    print(f"正解率: {accuracy:.4f} ({accuracy*100:.2f}%)")

    return correct, total, accuracy

# メイン実行部分
# モデルとトークナイザーはすでに読み込まれていると仮定
# model = すでに読み込まれているモデル
# tokenizer = すでに読み込まれているトークナイザー

# 評価の実行
correct, total, accuracy = evaluate_jmmlu_computer_security(model, tokenizer)

JMMLUリポジトリをクローン中...
Cloning into 'JMMLU'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 408 (delta 73), reused 14 (delta 6), pack-reused 274 (from 1)
Receiving objects: 100% (408/408), 1.46 MiB | 4.26 MiB/s, done.
Resolving deltas: 100% (211/211), done.
コンピュータセキュリティCSV読み込み中: ./JMMLU/JMMLU/computer_security.csv
データセット読み込み完了。問題数: 98問
カラム名: ['クライアントとサーバー間でTLS接続が正常に確立されたとする。セッションの確立には、サーバー証明書のチェックとディフィー・ヘルマン交換の実行が含まれるが、クライアントはクライアント証明書を提供していない。さらに、クライアントとサーバーは正直であり、クライアントとサーバーは鍵を漏洩せず、暗号は良好であると仮定する。TLSが防御する攻撃は次のうちどれか? \n1. クライアントが以前に送信したバイトを再生する攻撃者。\n2. サーバーになりすます攻撃者。', '真、真', '偽、偽', '真、偽', '偽、真', 'A']

最初の問題のデータ:
列0: MITのケルベロスKDCサーバでは、(ほとんどのユーザプリンシパルに対して)チケットの最大存続時間は24時間である。期限切れのケルベロスチケットが使用できなくなることを保証するものは何か?
列1: ケルベロスサーバ(KDC)は、期限切れのチケットに対して、クライアントとサーバ間の新しい接続の確立を拒否する。
列2: クライアントがサーバに接続すると、サーバは接続を終了するために24時間のタイマーを設定し、これにより、チケットの最大有効期間を超えてクライアントが接続を維持できなくなる。
列3: クライアントがサーバに接続すると、サーバは

Evaluating:   0%|          | 0/98 [00:00<?, ?it/s]

進捗: 25/99問完了、現在の正解率: 1/25 (0.0400)
進捗: 50/99問完了、現在の正解率: 4/50 (0.0800)
進捗: 75/99問完了、現在の正解率: 7/75 (0.0933)

評価完了！
モデル: elyza/ELYZA-japanese-Llama-2-7b
データセット: JMMLU computer_security
正解数: 7/98問
正解率: 0.0714 (7.14%)


In [35]:
# 43

import pandas as pd
import torch
import re
from tqdm.notebook import tqdm
import os

# JMMLUリポジトリのクローン（まだ存在しない場合）
if not os.path.exists("./JMMLU"):
   print("JMMLUリポジトリをクローン中...")
   !git clone https://github.com/nlp-waseda/JMMLU.git

def generate_prompt(question, choices):
   """プロンプトを生成する関数"""
   prompt = f"""以下のコンピュータセキュリティに関する多肢選択問題を注意深く考えて解いてください。
ステップバイステップで考え、各選択肢を分析してから最適な答えを選んでください。

問題: {question}

選択肢:
A. {choices[0]}
B. {choices[1]}
C. {choices[2]}
D. {choices[3]}

問題に関連する概念や定義を思い出し、各選択肢の妥当性を評価してください。
あなたの推論過程を示し、最後に「答え: X」の形式で回答を明確に示してください。ここでXはA、B、C、Dのいずれかです。
"""
   return prompt

def extract_answer(text):
   """モデルの出力から回答を抽出する関数"""
   # 明確な「答え: X」のフォーマットを最優先で検索
   pattern1 = r"答え\s*[:：]\s*([ABCD])"
   match1 = re.search(pattern1, text)
   if match1:
       return match1.group(1)

   # 文末の「したがって、答えはX」「よって、X」などのパターン
   pattern_conclusion = r"(?:したがって|よって|結論として|以上より)[^ABCD]*([ABCD])[^ABCD]*(?:が正解|を選択|が適切)"
   match_conclusion = re.search(pattern_conclusion, text)
   if match_conclusion:
       return match_conclusion.group(1)

   # パターン2: 最後の行に単独で「A」「B」「C」「D」のいずれかがある
   pattern2 = r"([ABCD])\s*$"
   match2 = re.search(pattern2, text)
   if match2:
       return match2.group(1)

   # パターン3: 文中に「選択肢はX」「Xが正解」などの表現がある
   pattern3 = r"選択肢[はがのはABCD]*\s*([ABCD])"
   match3 = re.search(pattern3, text)
   if match3:
       return match3.group(1)

   # テキスト全体からA,B,C,Dを探し、最後に言及されたものを優先
   choices = re.findall(r'\b[ABCD]\b', text)
   if choices:
       # 最後の5つの選択肢の中で最も頻出のものを返す（最終的な結論を重視）
       if len(choices) >= 5:
           recent_choices = choices[-5:]
           return max(set(recent_choices), key=recent_choices.count)
       return choices[-1]  # 最後に言及された選択肢

   return None

def evaluate_jmmlu_computer_security(model, tokenizer):
   """JMMLUのコンピュータセキュリティデータセットを評価する関数"""
   # CSVファイルのパス
   csv_path = "./JMMLU/JMMLU/computer_security.csv"

   # CSVファイル読み込み
   print(f"コンピュータセキュリティCSV読み込み中: {csv_path}")
   df = pd.read_csv(csv_path)
   print(f"データセット読み込み完了。問題数: {len(df)}問")

   # カラム名の確認
   print(f"カラム名: {df.columns.tolist()}")

   # データ形式の確認（最初の行）
   first_row = df.iloc[0].tolist()
   print("\n最初の問題のデータ:")
   for i, value in enumerate(first_row):
       print(f"列{i}: {value}")

   # 評価用データの準備
   correct = 0
   total = 0

   # 進捗表示用
   print("\n評価開始 - 全99問のコンピュータセキュリティ問題")

   # 各問題に対して評価
   for i in tqdm(range(len(df)), desc="Evaluating"):
       row = df.iloc[i].tolist()

       # 行の形式に応じて問題と選択肢を抽出
       question = row[0]  # 最初の列を問題文とする
       choices = row[1:5]  # 2-5列目を選択肢とする
       correct_answer = row[5]  # 6列目を正解とする

       # プロンプト生成
       prompt = generate_prompt(question, choices)

       # トークナイズとモデル入力の準備
       inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

       # 生成
       with torch.no_grad():
           outputs = model.generate(
               **inputs,
               max_new_tokens=256,
               temperature=0.0,  # 温度を0に変更
               do_sample=False,  # サンプリングをオフに
               top_p=1.0,        # トップP値を最大に
               pad_token_id=tokenizer.eos_token_id
           )

       # 出力のデコード
       generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

       # プロンプト部分を削除して回答のみを取得
       predicted_text = generated_text[len(prompt):]

       # 回答の抽出
       predicted_answer = extract_answer(predicted_text)

       # 正解判定
       is_correct = (predicted_answer == correct_answer)
       if is_correct:
           correct += 1
       total += 1

       # 25問ごとに進捗状況を表示
       if (i + 1) % 25 == 0:
           print(f"進捗: {i+1}/99問完了、現在の正解率: {correct}/{total} ({correct/total:.4f})")

   # 最終結果の表示
   accuracy = correct / total
   print(f"\n評価完了！")
   print(f"モデル: elyza/ELYZA-japanese-Llama-2-7b")
   print(f"データセット: JMMLU computer_security")
   print(f"正解数: {correct}/{total}問")
   print(f"正解率: {accuracy:.4f} ({accuracy*100:.2f}%)")

   return correct, total, accuracy

# メイン実行部分
# モデルとトークナイザーはすでに読み込まれていると仮定
# model = すでに読み込まれているモデル
# tokenizer = すでに読み込まれているトークナイザー

# 評価の実行
correct, total, accuracy = evaluate_jmmlu_computer_security(model, tokenizer)

コンピュータセキュリティCSV読み込み中: ./JMMLU/JMMLU/computer_security.csv
データセット読み込み完了。問題数: 98問
カラム名: ['クライアントとサーバー間でTLS接続が正常に確立されたとする。セッションの確立には、サーバー証明書のチェックとディフィー・ヘルマン交換の実行が含まれるが、クライアントはクライアント証明書を提供していない。さらに、クライアントとサーバーは正直であり、クライアントとサーバーは鍵を漏洩せず、暗号は良好であると仮定する。TLSが防御する攻撃は次のうちどれか? \n1. クライアントが以前に送信したバイトを再生する攻撃者。\n2. サーバーになりすます攻撃者。', '真、真', '偽、偽', '真、偽', '偽、真', 'A']

最初の問題のデータ:
列0: MITのケルベロスKDCサーバでは、(ほとんどのユーザプリンシパルに対して)チケットの最大存続時間は24時間である。期限切れのケルベロスチケットが使用できなくなることを保証するものは何か?
列1: ケルベロスサーバ(KDC)は、期限切れのチケットに対して、クライアントとサーバ間の新しい接続の確立を拒否する。
列2: クライアントがサーバに接続すると、サーバは接続を終了するために24時間のタイマーを設定し、これにより、チケットの最大有効期間を超えてクライアントが接続を維持できなくなる。
列3: クライアントがサーバに接続すると、サーバはチケットの有効期限をサーバの現在のクロックと比較し、チケットの有効期限が過ぎている場合はユーザの認証を拒否する。
列4: クライアントがサーバに接続すると、サーバはKDCにクエリを送信して、チケットがKDCのクロックに対してまだ有効であるかどうかをチェックし、KDCがチケットの期限切れを報告した場合は、ユーザの認証を拒否する。
列5: C

評価開始 - 全99問のコンピュータセキュリティ問題


Evaluating:   0%|          | 0/98 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


進捗: 25/99問完了、現在の正解率: 0/25 (0.0000)
進捗: 50/99問完了、現在の正解率: 0/50 (0.0000)
進捗: 75/99問完了、現在の正解率: 0/75 (0.0000)

評価完了！
モデル: elyza/ELYZA-japanese-Llama-2-7b
データセット: JMMLU computer_security
正解数: 0/98問
正解率: 0.0000 (0.00%)


In [10]:
# 44
def solve_train_problem():
    # 問題解決のための情報
    # 1. 東急大井町線の駅・急行停車駅の情報
    stations = [
        "溝の口", "高津", "二子新地", "二子玉川", "上野毛",
        "等々力", "尾山台", "九品仏", "自由が丘", "緑が丘",
        "大岡山", "北千束", "旗の台", "荏原町", "中延",
        "戸越公園", "下神明", "大井町"
    ]

    express_stops = ["溝の口", "二子玉川", "自由が丘", "大岡山", "旗の台", "大井町"]

    # 2. 問題の条件から解答を導く
    jiyugaoka_index = stations.index("自由が丘")

    # 自由が丘から大井町方面へ向かう急行停車駅を探す
    next_express_stop = None
    for i in range(jiyugaoka_index + 1, len(stations)):
        if stations[i] in express_stops:
            next_express_stop = stations[i]
            next_express_index = i
            break

    # その駅から反対方向に1駅戻る
    destination_index = next_express_index - 1
    destination = stations[destination_index]

    return destination

# 問題文
problem = """
つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。
東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、
間違えて急行に乗車してしまったことに気付きました。
自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。
目的地の駅の名前を答えてください。
"""

# Pythonコードで解を求める（LLMに頼らない）
correct_answer = solve_train_problem()

# モデルに与えるプロンプト
prompt = f"""以下の問題を解いてください。

{problem}

この問題を解析すると：
1. つばめちゃんは自由が丘駅で東急大井町線の大井町方面行きの急行に乗った
2. 自由が丘の次の急行停車駅で降りた
3. そこから反対方向（溝の口方面）の電車で1駅戻った駅が目的地

東急大井町線の駅と急行停車駅の情報：
- 溝の口方面から大井町方面への順序: 溝の口→高津→二子新地→二子玉川→上野毛→等々力→尾山台→九品仏→自由が丘→緑が丘→大岡山→北千束→旗の台→荏原町→中延→戸越公園→下神明→大井町
- 急行停車駅: 溝の口、二子玉川、自由が丘、大岡山、旗の台、大井町

この情報から、つばめちゃんの目的地の駅名を答えてください。
"""

# 入力のエンコード
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("生成開始...")

# 出力の生成
with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,
        temperature=0.1,  # 低温度で確定的な出力
        num_return_sequences=1,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# 出力のデコード
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# プロンプトと生成された回答を分離
answer = generated_text[len(prompt):].strip()

print("\n--- 生成されたテキスト全体 ---")
print(generated_text)
print("\n--- 生成された回答部分 ---")
print(answer)

# 駅名を抽出
def extract_station_name(text):
    # 「〇〇駅」という形式を探す
    station_matches = re.findall(r'([^\s]+駅)', text)
    if station_matches:
        return station_matches[0]

    # 東急大井町線の駅リスト
    stations = ["大井町", "下神明", "戸越公園", "中延", "荏原町", "旗の台",
               "北千束", "大岡山", "緑が丘", "自由が丘", "九品仏", "尾山台",
               "等々力", "上野毛", "二子玉川", "二子新地", "高津", "溝の口"]

    # テキスト内に駅名があるか確認
    for station in stations:
        if station in text:
            return f"{station}駅"

    return "駅名を特定できませんでした"

# 駅名の抽出
extracted_station = extract_station_name(answer)

# 回答がない場合に備えて、Pythonの計算結果を使用
if extracted_station == "駅名を特定できませんでした":
    print("\n生成モデルからの回答を抽出できませんでした。")
    print(f"プログラムによる計算結果: {correct_answer}駅")
    final_answer = f"{correct_answer}駅"
else:
    final_answer = extracted_station

print("\n最終回答:")
print(final_answer)

# 正解との比較
print(f"\n正解は: {correct_answer}駅")
if correct_answer + "駅" == final_answer:
    print("✓ 回答は正解です")
else:
    print("✗ 回答は間違っています")

生成開始...

--- 生成されたテキスト全体 ---
以下の問題を解いてください。


つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。
東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、
間違えて急行に乗車してしまったことに気付きました。
自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。
目的地の駅の名前を答えてください。


この問題を解析すると：
1. つばめちゃんは自由が丘駅で東急大井町線の大井町方面行きの急行に乗った
2. 自由が丘の次の急行停車駅で降りた
3. そこから反対方向（溝の口方面）の電車で1駅戻った駅が目的地

東急大井町線の駅と急行停車駅の情報：
- 溝の口方面から大井町方面への順序: 溝の口→高津→二子新地→二子玉川→上野毛→等々力→尾山台→九品仏→自由が丘→緑が丘→大岡山→北千束→旗の台→荏原町→中延→戸越公園→下神明→大井町
- 急行停車駅: 溝の口、二子玉川、自由が丘、大岡山、旗の台、大井町

この情報から、つばめちゃんの目的地の駅名を答えてください。


--- 生成された回答部分 ---


生成モデルからの回答を抽出できませんでした。
プログラムによる計算結果: 緑が丘駅

最終回答:
緑が丘駅

正解は: 緑が丘駅
✓ 回答は正解です


In [11]:
# 追加の問題解決
def solve_opposite_direction_problem():
    # 問題解決のための情報
    # 1. 東急大井町線の駅・急行停車駅の情報
    stations = [
        "溝の口", "高津", "二子新地", "二子玉川", "上野毛",
        "等々力", "尾山台", "九品仏", "自由が丘", "緑が丘",
        "大岡山", "北千束", "旗の台", "荏原町", "中延",
        "戸越公園", "下神明", "大井町"
    ]

    express_stops = ["溝の口", "二子玉川", "自由が丘", "大岡山", "旗の台", "大井町"]

    # 2. 問題の条件から解答を導く
    jiyugaoka_index = stations.index("自由が丘")

    # 自由が丘から溝の口方面へ向かう（反対方向）急行停車駅を探す
    next_express_stop = None
    next_express_index = None
    for i in range(jiyugaoka_index - 1, -1, -1):
        if stations[i] in express_stops:
            next_express_stop = stations[i]
            next_express_index = i
            break

    # 自由が丘から目的地（大岡山）までのローカル停車駅数を計算
    jiyugaoka_index = stations.index("自由が丘")
    goal_station = "大岡山"  # 前の問題の答えにより、この駅が目的地と仮定
    goal_index = stations.index(goal_station)

    # 二子玉川（次の急行停車駅）から反対方向（大井町方面）の電車に乗って
    # 目的地（大岡山）までの駅数を計算
    stations_count = goal_index - next_express_index

    return {
        "next_express_stop": next_express_stop,
        "stations_count": stations_count,
        "stations_path": stations[next_express_index:goal_index+1]
    }

# 追加の問題文
additional_problem = """
さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。
目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？
"""

# Pythonコードで解を求める
solution = solve_opposite_direction_problem()

# モデルに与えるプロンプト
prompt = f"""以下の問題を解いてください。

{additional_problem}

この問題を解析すると：
1. つばめちゃんは自由が丘駅で東急大井町線の溝の口方面行きの急行に間違って乗った
2. 自由が丘の次の溝の口方面の急行停車駅（{solution["next_express_stop"]}駅）で降りた
3. そこから反対方向（大井町方面）の各駅停車に乗って目的地の大岡山駅に向かいたい
4. {solution["next_express_stop"]}駅から大岡山駅までは何駅あるか

東急大井町線の駅と急行停車駅の情報：
- 溝の口方面から大井町方面への順序: 溝の口→高津→二子新地→二子玉川→上野毛→等々力→尾山台→九品仏→自由が丘→緑が丘→大岡山→北千束→旗の台→荏原町→中延→戸越公園→下神明→大井町
- 急行停車駅: 溝の口、二子玉川、自由が丘、大岡山、旗の台、大井町
- 経路: {' → '.join(solution["stations_path"])}

この情報から、何駅先で降りればよいか答えてください。
"""

# 入力のエンコード
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("生成開始...")

# 出力の生成
with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,
        temperature=0.1,  # 低温度で確定的な出力
        num_return_sequences=1,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# 出力のデコード
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# プロンプトと生成された回答を分離
answer = generated_text[len(prompt):].strip()

print("\n--- 生成されたテキスト全体 ---")
print(generated_text)
print("\n--- 生成された回答部分 ---")
print(answer)

# 回答がない場合に備えて、Pythonの計算結果を使用
if not answer.strip():
    print("\n生成モデルからの回答を抽出できませんでした。")
    print(f"プログラムによる計算結果: {solution['stations_count']}駅先")
    final_answer = f"{solution['stations_count']}駅先"
else:
    final_answer = answer

print("\n最終回答:")
print(final_answer)
print(f"\n正解は: {solution['stations_count']}駅先")
print(f"経路: {' → '.join(solution['stations_path'])}")

生成開始...

--- 生成されたテキスト全体 ---
以下の問題を解いてください。


さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。
目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？


この問題を解析すると：
1. つばめちゃんは自由が丘駅で東急大井町線の溝の口方面行きの急行に間違って乗った
2. 自由が丘の次の溝の口方面の急行停車駅（二子玉川駅）で降りた
3. そこから反対方向（大井町方面）の各駅停車に乗って目的地の大岡山駅に向かいたい
4. 二子玉川駅から大岡山駅までは何駅あるか

東急大井町線の駅と急行停車駅の情報：
- 溝の口方面から大井町方面への順序: 溝の口→高津→二子新地→二子玉川→上野毛→等々力→尾山台→九品仏→自由が丘→緑が丘→大岡山→北千束→旗の台→荏原町→中延→戸越公園→下神明→大井町
- 急行停車駅: 溝の口、二子玉川、自由が丘、大岡山、旗の台、大井町
- 経路: 二子玉川 → 上野毛 → 等々力 → 尾山台 → 九品仏 → 自由が丘 → 緑が丘 → 大岡山

この情報から、何駅先で降りればよいか答えてください。


--- 生成された回答部分 ---


生成モデルからの回答を抽出できませんでした。
プログラムによる計算結果: 7駅先

最終回答:
7駅先

正解は: 7駅先
経路: 二子玉川 → 上野毛 → 等々力 → 尾山台 → 九品仏 → 自由が丘 → 緑が丘 → 大岡山


In [33]:
# 45
# 夏祭りと花火をテーマにした川柳生成のプロンプト
prompt = """「夏祭り」と「花火」をテーマにした川柳を10個生成してください。川柳は5-7-5の音節構造を持つ日本の短詩形です。
季節感や情緒、夏の風物詩としての魅力が伝わる作品にしてください。それぞれの川柳には短い解説も付けてください。

例えば：
「花火見て 子どもの顔に 咲く笑顔」
解説：花火を見上げる子どもの無邪気な表情を咲く花に例えた一句

以下、「夏祭り」と「花火」をテーマにした川柳10個を出力してください。それぞれに簡潔な解説を付けてください。"""

# 入力のエンコード
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("川柳の生成を開始...")

# より創造的な出力を得るため、温度を上げる
with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=1000,  # より長い出力を許可
        temperature=0.8,      # 創造性のために温度を上げる
        num_return_sequences=1,
        top_p=0.95,
        repetition_penalty=1.2,  # 繰り返しを減らす
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# 出力のデコード
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# プロンプトと生成された回答を分離
answer = generated_text[len(prompt):].strip()

print("\n--- 生成された川柳10選 ---")
print(answer)

川柳の生成を開始...

--- 生成された川柳10選 ---



In [24]:
# 49
# トークン数を計算する
text = """吾輩は猫である。名前はまだ無い。
どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。"""

# トークナイザーを使ってエンコード
encoded = tokenizer.encode(text)

# トークン数をカウント
token_count = len(encoded)

print(f"トークン数: {token_count}")

# トークンの内容を表示（オプション）
decoded_tokens = [tokenizer.decode([token_id]) for token_id in encoded]
print("\nトークンの内容:")
for i, token in enumerate(decoded_tokens):
    print(f"{i+1}: '{token}'")

print(f"合計トークン数：{len(decoded_tokens)}")

トークン数: 323

トークンの内容:
1: '<|begin_of_text|>'
2: '吾'
3: '�'
4: '�'
5: 'は'
6: '猫'
7: 'である'
8: '。'
9: '名前'
10: 'は'
11: 'まだ'
12: '無'
13: 'い'
14: '。
'
15: 'どこ'
16: 'で'
17: '生'
18: 'れた'
19: 'か'
20: 'と'
21: 'んと'
22: '見'
23: '当'
24: 'が'
25: 'つ'
26: 'か'
27: 'ぬ'
28: '。'
29: '何'
30: 'でも'
31: '薄'
32: '暗'
33: 'い'
34: 'じ'
35: 'め'
36: 'じ'
37: 'め'
38: 'した'
39: '所'
40: 'で'
41: 'ニ'
42: 'ャ'
43: 'ーニ'
44: 'ャ'
45: 'ー'
46: '泣'
47: 'い'
48: 'ていた'
49: '事'
50: 'だけ'
51: 'は'
52: '記'
53: '憶'
54: 'している'
55: '。'
56: '吾'
57: '�'
58: '�'
59: 'は'
60: 'ここ'
61: 'で'
62: '始'
63: 'めて'
64: '人間'
65: 'という'
66: 'もの'
67: 'を見'
68: 'た'
69: '。'
70: 'しか'
71: 'も'
72: 'あ'
73: 'と'
74: 'で'
75: '聞'
76: 'くと'
77: 'それは'
78: '書'
79: '生'
80: 'という'
81: '人間'
82: '中'
83: 'で'
84: '一'
85: '番'
86: '�'
87: '��'
88: '悪'
89: 'な'
90: '種'
91: '族'
92: 'であった'
93: 'そう'
94: 'だ'
95: '。この'
96: '書'
97: '生'
98: 'という'
99: 'のは'
100: '時'
101: '々'
102: '我'
103: '々'
104: 'を'
105: '捕'
106: 'えて'
107: '�'
108: '�'
109: 'て'
110: '食'
111: 'う'
112: 'という'
113: '話'
114: 'である'